# Extraction

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import textdistance as td

# Overlap similarity using other similarity function
def overlap(words1, words2, simFunc=lambda x, y: 1 if x == y else 0, threshold=1):    
    intersec = 0

    words1, words2 = set(words1), set(words2)

    lenWords1, lenWords2 = len(words1), len(words2)

    if lenWords1 > lenWords2:
        aux = words1
        words1 = words2
        words2 = aux

    for token1 in words1:
        for token2 in words2:
            if simFunc(token1, token2) >= threshold:
                words2.remove(token2)   # Not necessary
                intersec += 1
                break

    return intersec / min(lenWords1, lenWords2)

def nameSimFunc(wine1, wine2, simFunc=td.jaccard.normalized_similarity):
    # Remove vintage
    name1 = wine1['name'].replace(str(wine1['vintage']), '').replace('  ', ' ').strip()
    name2 = wine2['name'].replace(str(wine2['vintage']), '').replace('  ', ' ').strip()

    # Remove producer
    name1 = name1.replace(wine1['producer'], '').replace('  ', ' ').strip()
    name1 = name1.replace(wine2['producer'], '').replace('  ', ' ').strip()
    name2 = name2.replace(wine1['producer'], '').replace('  ', ' ').strip()
    name2 = name2.replace(wine2['producer'], '').replace('  ', ' ').strip()

    # Remove variety
    for variety in wine1['variety']:
        if variety:
            name1 = name1.replace(variety, '').replace('  ', ' ').strip()
            name2 = name2.replace(variety, '').replace('  ', ' ').strip()
    if 'variety' in wine2.keys():
        for variety in wine2['variety']:
            if variety:
                name1 = name1.replace(variety, '').replace('  ', ' ').strip()
                name2 = name2.replace(variety, '').replace('  ', ' ').strip()

    # Remove region
    for region in wine1['region'].split(', '):
        name1 = name1.replace(region, '').replace('  ', ' ').strip()
        name2 = name2.replace(region, '').replace('  ', ' ').strip()
    for region in wine2['region'].split(', '):
        name1 = name1.replace(region, '').replace('  ', ' ').strip()
        name2 = name2.replace(region, '').replace('  ', ' ').strip()
    
    if not name1 or not name2:
        name1 = wine1['producer'] + ' ' + (name1 + ' ' if name1 else '') + \
                wine1['region'].split(', ')[0]
        name1 = name1.strip()

        name2 = wine2['producer'] + ' ' + (name2 + ' ' if name2 else '') + \
                wine2['region'].split(', ')[0]
        name2 = name2.strip()
    
    return simFunc(name1.split(), name2.split())

def ratingSimFunc(x, rate=0.1):    
    dist = x['dist']
    for i in range(1, x['count_matching_ratings']+1):
        dist = 1 - (1-dist)*np.exp(-i*rate)
    return dist

def insertWine(wineInfo, conn, cursor):
    # ----- Wine Deduplication -----
    sql = f"SELECT wine.id, wine.region, wine.producer, wine.name, wine.vintage, wine_variety.name AS variety"

    if 'ratings' in wineInfo.keys() and len(wineInfo['ratings']):
        sql += f", (SELECT COUNT(*) FROM rating WHERE wine = wine.id AND ("
        for rating in wineInfo['ratings']:
            sql += f"(rating.type = '{rating['name']}' AND rating.value = {rating['rating']}) OR " 
        sql = sql[:-3] + ")) AS count_matching_ratings"

    sql += f" FROM wine LEFT JOIN wine_variety ON wine.id = wine_variety.wine"

    sql += f" WHERE color='{wineInfo['color']}' \
              AND size={wineInfo['size']} \
              AND vintage={wineInfo['vintage']}"
    
    if 'ratings' in wineInfo.keys() and len(wineInfo['ratings']):
        sql += f" AND wine.id NOT IN (SELECT wine FROM rating WHERE "
        for rating in wineInfo['ratings']:
            sql += f"(rating.type='{rating['name']}' AND rating.value != {rating['rating']}) OR "
        sql = sql[:-4] + ')'
    
        wines = pd.read_sql(sql, conn)
        wines = wines.groupby('id region producer name vintage count_matching_ratings'.split())['variety'].apply(list).reset_index(name='variety')
    else:
        wines = pd.read_sql(sql, conn)
        wines = wines.groupby('id region producer name vintage'.split())['variety'].apply(list).reset_index(name='variety')

    if wines.shape[0] > 0:
        producerSim = lambda x: td.levenshtein.normalized_similarity(x['producer'], wineInfo['producer'])
        wines['sim_prod'] = wines.apply(producerSim, axis=1)
        # wines = wines.loc[(wines['sim_prod'] > 0.6)]

    if wines.shape[0] > 0:        
        regionSim = lambda x: overlap(x['region'].split(', '), wineInfo['region'].split(', '), 
                                      td.levenshtein.normalized_similarity, 0.8)
        wines['sim_region'] = wines.apply(regionSim, axis=1)
        # wines = wines.loc[(wines['sim_region'] > 0.8)]
    
    if wines.shape[0] > 0:
        nameSim = lambda x: nameSimFunc(x, wineInfo, td.jaccard.normalized_similarity)
        wines['sim_name'] = wines.apply(nameSim, axis=1)
        # wines = wines.loc[(wines['sim_name'] > 0.8)]
    
    if wines.shape[0] > 0:
        dist = lambda x: np.sqrt((x['sim_region']**2 + x['sim_prod']**2 + x['sim_name']**2)/3)
        wines['dist'] = wines.apply(dist, axis=1)

        if 'ratings' in wineInfo.keys() and len(wineInfo['ratings']):
            wines['dist'] = wines.apply(lambda x: ratingSimFunc(x, 0.1), axis=1)

        wines = wines.loc[(wines['dist'] > 0.9)]

    # If wine was not found, insert
    if wines.shape[0] > 0:
        print(f"{wineInfo['name']} = {wines['name'].values[0]}: ({wines['sim_region'].values[0]:.1f}, {wines['sim_prod'].values[0]:.1f}, {wines['sim_name'].values[0]:.1f}) -> {wines['dist'].values[0]:.1f}")

        wines = wines.sort_values(['dist'], ascending=False)
        wine_id = wines['id'].values[0]
    else:
        if 'producer' in wineInfo.keys():
            cursor.execute(f"INSERT INTO wine (color, name, producer, region, vintage, size) \
                             VALUES ('{wineInfo['color']}', '{wineInfo['name']}', '{wineInfo['producer']}', \
                                     '{wineInfo['region']}', {wineInfo['vintage']}, {wineInfo['size']})")
            cursor.execute(f"SELECT id FROM wine WHERE color='{wineInfo['color']}' AND name='{wineInfo['name']}' AND \
                                                       region='{wineInfo['region']}' AND vintage={wineInfo['vintage']} AND \
                                                       size={wineInfo['size']} AND producer='{wineInfo['producer']}'")
        else:
            cursor.execute(f"INSERT INTO wine (color, name, region, vintage, size) \
                             VALUES ('{wineInfo['color']}', '{wineInfo['name']}', '{wineInfo['region']}', \
                                     {wineInfo['vintage']}, {wineInfo['size']})")
            cursor.execute(f"SELECT id FROM wine WHERE color='{wineInfo['color']}' AND name='{wineInfo['name']}' AND \
                                                       region='{wineInfo['region']}' AND vintage={wineInfo['vintage']} AND \
                                                       size={wineInfo['size']}")
        wine_id = cursor.fetchone()[0]

        if 'variety' in wineInfo.keys():
            # Wine Variety
            for variety in wineInfo['variety']:
                cursor.execute(f"INSERT INTO wine_variety VALUES ({wine_id}, '{variety}')")

    # ----- Dependent Attributes -----

    # Images
    for image in wineInfo['image']:
        cursor.execute(f"INSERT INTO image (link, wine) \
                         SELECT * FROM (SELECT '{image}' AS link, {wine_id} AS wine) AS tmp \
                         WHERE NOT EXISTS (SELECT * FROM image WHERE link='{image}' AND wine={wine_id}) LIMIT 1")

    # Ratings
    if 'ratings' in wineInfo.keys():
        for rating in wineInfo['ratings']:
            cursor.execute(f"INSERT INTO rating (type, value, wine) \
                             SELECT * FROM (SELECT '{rating['name']}' AS type, {rating['rating']} AS value, {wine_id} AS wine) AS tmp \
                             WHERE NOT EXISTS (SELECT * FROM rating WHERE wine={wine_id} AND type='{rating['rating']}')")

    if 'price' in wineInfo.keys():
        # Price
        cursor.execute(f"INSERT INTO price (value, currency, store, wine) \
                         VALUES ({wineInfo['price']['value']}, '{wineInfo['price']['currency']}', \
                                 '{wineInfo['store']}', {wine_id})")

    conn.commit()

In [3]:
import mysql.connector
import warnings
import json

warnings.filterwarnings('ignore')

# ----- MySQL Connection -----
conn = mysql.connector.connect(host='localhost',
                               database='thewinegame',
                               user='user',
                               password='password',
                               port=6033)
cursor = conn.cursor(buffered=True)

with open('./db/data_bestOfWines.json', 'r') as file:
    allWines = json.load(file)

for wineInfo in allWines:
    print(wineInfo['name'])
    # try:
    insertWine(wineInfo, conn, cursor)
    # except Exception as err:
    #     print(err)

conn.close()

Adversity Cabernet Sauvignon Erba Vineyard 2019
Albert Bichot Clos De Vougeot Domaine De Clos Frantin 2019
Almaviva Puente Alto 2019
Almaviva Puente Alto 2019 = Almaviva Puente Alto 2019: (1.0, 1.0, 1.0) -> 1.0
Amuse Bouche Leroy Neiman 2019
Amuse Bouche Proprietary Red 2019
Au Sommet Cabernet Sauvignon Atlas Peak 2019
Beaulieu Vineyard Georges De Latour Private Reserve Cabernet Sauvignon 2019
Beaulieu Vineyard Georges De Latour Private Reserve Cabernet Sauvignon 2019 = Beaulieu Vineyard Georges De Latour Private Reserve Cabernet Sauvignon 2019: (1.0, 1.0, 1.0) -> 1.0
Beaux Freres Pinot Noir The Belles Soeurs 2019
Bibi Graetz Colore 2019
Camarcanda 2019
Carte Blanche Cabernet Sauvignon Beckstoffer Missouri Hopper Vineyard 2019
Carte Blanche The Mark 2019
Catena Zapata Adrianna Vineyard Mundus Bacillus Terrae 2019
Ceretto Barbaresco Bricco Asili 2019
Ceretto Barbaresco Bricco Asili 2019 = Ceretto Barbaresco Bricco Asili 2019: (1.0, 1.0, 1.0) -> 1.0
Cesare Bussolo Barolo Fossati 2019
Cha

In [4]:
import json

with open('./db/data_bestOfWines.json', 'r') as file:
    allWines = json.load(file)
print(len(allWines))

107


In [30]:
conn.close()

## Test Extraction

In [5]:
import mysql.connector

# ----- MySQL Connection -----
conn = mysql.connector.connect(host='localhost',
                               database='thewinegame',
                               user='user',
                               password='password',
                               port=6033)
cursor = conn.cursor(buffered=True)

wineInfo = {
    'color': 'Red',
    'store': 'wine.com',
    'name': 'Adelaida Viking Estate Vineyard Signature Cabernet Sauvignon',
    'vintage': 2019,
    'producer': 'Adelaida Winery',
    'variety': ['Cabernet Sauvignon'],
    'region': 'Adelaida District, Paso Robles, Central Coast, California',
    'price': {'currency': 'euro', 'value': 154.95},
    'ratings': [{'name': 'Jeb Dunnuck', 'rating': 96}, {'name': 'Robert Parkers Wine Advocate', 'rating': 94}],
    'size': 750,
    'image': ['https://www.wine.com/product/images/w_600,h_600,c_fit,q_auto:good,fl_progressive/arqsadblqhvcmeb5fbvb.jpg']
}

# if not insertWine(wineInfo, conn, cursor):
#     conn.close()
insertWine(wineInfo, conn, cursor)
conn.close()

C:\Users\eduar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Adelaida Winery Adelaida Viking Estate Vineyard Signature Cabernet Sauvignon = Adelaida Winery Adelaida Viking Estate Vineyard Signature Cabernet Sauvignon: (1.0, 1.0, 1.0)


In [4]:
conn.close()

# Scrappers

In [22]:
import json

def writeToJson(wineInfo, fileName):
    # Escrever de volta no arquivo JSON
    with open(f'./db/{fileName}', 'w') as file:
        json.dump(wineInfo, file)

## wine.com

In [20]:
import requests
import time
from datetime import timedelta
from sys import stdout
from bs4 import BeautifulSoup
import mysql.connector
import warnings

warnings.filterwarnings('ignore')

# ----- MySQL Connection -----
conn = mysql.connector.connect(host='localhost',
                               database='thewinegame',
                               user='user',
                               password='password',
                               port=6033)
cursor = conn.cursor(buffered=True)

baseUrl = 'https://www.wine.com'

# page = requests.get(f'{baseUrl}/list/wine/red-wine/7155-124?showOutOfStock=true')
page = requests.get('https://www.wine.com/list/wine/red-wine/standard-750ml/vintage-2019/7155-124-266-384?showOutOfStock=true&sortBy=wineryAToZ&pricemin=100&pricemax=200')
soup = BeautifulSoup(page.text, 'html.parser')

numItemsText = soup.find('h1', class_='listLearnAboutHead')
numItems = int(numItemsText.find('span', class_='count').text.replace(',', ''))

allWines = []

cont = 0
while True:
    soup = BeautifulSoup(page.text, 'html.parser')

    wineList = soup.find('ul', class_='listGridLayout_list')
    wineElements = wineList.find_all('li', class_='listGridLayout_listItem')

    # iterate <li> items
    for wineElem in wineElements:
        # Try <5 times to load the product page
        trials = 0
        while trials < 5:
            try:
                wineInfo = { 'color': 'Red', 'store': 'wine.com' }   # Iterate over red, white, ...

                # ----- Getting info from main page -----
                # Name and vintage  
                name = wineElem.find('span', itemprop='name').text
                vintage = name[-4:]
                wineInfo['name'] = name.replace("'", '').strip()
                wineInfo['name'] = wineInfo['name'].replace('-', ' ').title()   # -

                wineInfo['vintage'] = int(vintage)

                # Variety
                variety = wineElem.find('span', class_='listGridItemOrigin_varietal').text
                wineInfo['variety'] = [variety.replace("'", '').strip().title()]

                # Origin
                region = wineElem.find('span', class_='listGridItemOrigin_text').text
                wineInfo['region'] = region.replace("'", '').replace('-', ' ').strip().title()

                # Price
                price = wineElem.find('meta', itemprop='price')['content']
                wineInfo['price'] = {'currency': 'dolar', 'value': float(price)}

                # Ratings
                ulRatings = wineElem.find('ul', class_='wineRatings_list')
                if ulRatings:
                    ratings = []
                    for ratingElem in ulRatings.find_all('li'):
                        ratingName = ratingElem['title'][:-16].replace("'", '').strip().title()
                        ratingValue = ratingElem.find('span', class_='wineRatings_rating').text
                        
                        rating = {'name': ratingName, 'rating': int(ratingValue)}

                        ratings.append(rating)

                    wineInfo['ratings'] = ratings

                # ----- Getting info from wine page -----
                
                # Get wine detail URL
                a = wineElem.find('a', class_='event_productClick', href=True)
                wineUrl = f"{baseUrl}{a['href']}"

                # Open wine page
                winePage = requests.get(wineUrl)
                wineSoup = BeautifulSoup(winePage.text, 'html.parser')
                
                # Getting info from product details table
                productDetails = wineSoup.find('section', class_='pipProdDetails')
                titles = productDetails.find_all('div', class_='pipProdDetails_title')
                values = productDetails.find_all('div', class_='pipProdDetails_name')
                for i, title in enumerate(titles):
                    if title.text.strip() == 'Size':    # Size
                        wineInfo['size'] = int(values[i].text[:-2])
                    elif title.text.strip() == 'Producer':  # Producer
                        wineInfo['producer'] = values[i].text.replace("'", '').strip()
                        wineInfo['producer'] = wineInfo['producer'].replace('-', ' ').title()
                
                # Images
                divImage = wineSoup.find('div', class_='pipProdThumbs')
                images = []
                for image in divImage.find_all('img'):
                    imageUrl = image['src'].split('/')[-1]
                    images.append('https://www.wine.com/product/images/w_600,h_600,c_fit,q_auto:good,fl_progressive/' + imageUrl)
                
                wineInfo['image'] = images

                print(wineInfo['name'])

                # if 'producer' in wineInfo.keys():
                #     wineInfo['name'] = wineInfo['name'].replace(wineInfo['producer'], '').strip()

                # Print wine
                # for key, value in wineInfo.items():
                #     if key == 'name':
                #         print(f"{key}: {value}")
                

                # insertWine(wineInfo, conn, cursor)
                allWines.append(wineInfo)
                
                break
            except AttributeError as err:
                if "'NoneType' object has no attribute" not in str(err):
                    print(err)
                trials += 1
                continue
        else:
            print(f"FAIL!") 

    # Load next page
    nextPageBtn = soup.find('a', class_='listPageNextUrl')
    if nextPageBtn:
        nextPageUrl = nextPageBtn['href']
        page = requests.get(nextPageUrl)
    else:
        break

    cont += 1

writeToJson(allWines)

conn.close()

00 Wines Vgr Pinot Noir 2019
Aalto Ps 2019
Accendo Cellars Laurea Red Wine 2019
Addax Tench Vineyard Cabernet Sauvignon 2019
Adelaida Viking Estate Vineyard Signature Cabernet Sauvignon 2019
FAIL!
Albert Bichot Aloxe Corton Les Fournieres Premier Cru Domaine Du Pavillon Monopole 2019
Albert Bichot Vosne Romanee Domaine Du Clos Frantin 2019
Albert Bichot Volnay Les Santenots Premier Cru Domaine Du Pavillon 2019
Albert Bichot Gevrey Chambertin Les Evocelles 2019
Albert Bichot Vosne Romanee Les Rouges Premier Cru Domaine Du Clos Frantin 2019
Albert Bichot Nuits Saint Georges Premier Cru Chateau Gris Monopole 2019
Albert Bichot Morey St Denis Les Sorbets Premier Cru 2019
Albert Bichot Chambolle Musigny Les Chabiots Premier Cru 2019
Albert Bichot Aloxe Corton Clos Des Marechaudes Premier Cru Domaine Du Pavillon Monopole 2019
Albert Bichot Gevrey Chambertin 2019
Albert Bichot Pommard Les Rugiens Premier Cru Domaine Du Pavillon 2019
Aldo Conterno Barolo Bussia 2019
Alejandro Bulgheroni Lithol

In [18]:
conn.close()

## Best of Wines

In [21]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
import time
import mysql.connector

# ----- MySQL Connection -----
# conn = mysql.connector.connect(host='localhost',
#                                database='thewinegame',
#                                user='user',
#                                password='password',
#                                port=6033)
# cursor = conn.cursor(buffered=True)

options = webdriver.ChromeOptions() 
options.headless = True 

driver = webdriver.Chrome('./chromedriver.exe', options=options)
driver2 = webdriver.Chrome('./chromedriver.exe', options=options)

baseUrl = 'https://bestofwines.com'
url = f"{baseUrl}/wine/?type=2%2C3%2C4%2C11%2C12%2C17&view=grid&types%5B%5D=2&years%5B%5D=2019&price_min=%E2%82%AC%20100%2C-&price_max=%E2%82%AC%20200%2C-&volumes%5B%5D=3&order_dir=asc&order_by=suppliers.supplier"

driver.get(url)

lastHeight = driver.execute_script('return document.body.scrollHeight')

allWines = []

finish = False
while not finish:
	driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
	time.sleep(1)
	newHeight = driver.execute_script('return document.body.scrollHeight')
	if newHeight == lastHeight: 
		break 

	lastHeight == newHeight

	wineList = driver.find_elements(By.XPATH, "//div[@id='wines-result']/a[@class='tile-wrapper show-wine']")
	# Iterate wines
	for i, wineElem in enumerate(wineList):
		wineInfo = {}
		
		# Store name
		wineInfo['store'] = 'Best of Wines'

		# Make soup for wine page
		wineUrl = wineElem.get_attribute('href')

		# Price
		try:
			exVatPrice = wineElem.find_element(By.XPATH, ".//span[@class='prices']/span[@class='price sale']").text
		except:
			exVatPrice = wineElem.find_element(By.XPATH, ".//span[@class='prices']/span[@class='price ']").text

		if exVatPrice != "on request":
			wineInfo['price'] = {}
			exVatPrice = exVatPrice[1:-8].replace('â\x82¬', '')
			exVatPrice = exVatPrice.replace('.', '').replace(',', '.')
			wineInfo['price'] = {'value': float(exVatPrice), 'currency': 'euro'}

		driver2.get(wineUrl)

		# Name
		spanInfo = wineElem.find_element(By.XPATH, ".//span[@class='title']")
		wineInfo['name'] = spanInfo.find_element(By.XPATH, ".//h4").text.replace("'", '').strip()

		try:
			# Iterate wine spec table
			wineSpecDiv = driver2.find_element(By.XPATH, ".//div[@id='wine-specs']")
		except:
			continue

		specList = wineSpecDiv.find_elements(By.XPATH, './/tr')
		for specElem in specList:
			specLabels = specElem.find_elements(By.XPATH, './/td')
			if specLabels[0].text == 'TYPE':			# Color
				wineInfo['color'] = specLabels[1].text.strip()	
			elif specLabels[0].text == 'BRAND':			# Producer
				wineInfo['producer'] = specLabels[1].text.replace("'", '')
				wineInfo['producer'] = wineInfo['producer'].replace('-', ' ').strip()
				wineInfo['producer'] = wineInfo['producer'].title()
			elif specLabels[0].text == 'VINTAGE':		# Vintage
				wineInfo['vintage'] = specLabels[1].text
			elif specLabels[0].text == 'VOLUME':		# Size
				size = float(specLabels[1].text.replace(',', '.'))
				wineInfo['size'] = int(size*1000)
			elif specLabels[0].text == 'COUNTRY':		# Region
				wineInfo['region'] = specLabels[1].text.strip()
			elif specLabels[0].text == 'REGION':
				region = specLabels[1].text.strip()
				region = region.split(', ')[::-1]
				region.append(wineInfo['region'])
				region = ', '.join(region)
				wineInfo['region'] = region
			elif specLabels[0].text == 'GRAPE':			# Variety
				variety = specLabels[1].text.split(',')
				variety = [v.strip() for v in variety if v.strip() != '']
				wineInfo['variety'] = variety
	
		wineInfo['region'] = wineInfo['region'].title()

		# Vintage formatting
		if wineInfo['vintage'].isdigit():
			wineInfo['vintage'] = int(wineInfo['vintage'])
		else:
			del wineInfo['vintage']
			continue

		# Image
		aImg = driver2.find_element(By.XPATH, ".//div[@class='wine-img wine-tab wine-tab-0']/a")
		wineInfo['image'] = [aImg.get_attribute('href')]

		# Ratings
		divRatings = driver2.find_element(By.XPATH, ".//div[@id='wine-reviews']")
		if divRatings:
			ratingTitles = divRatings.find_elements(By.XPATH, ".//h4")
			ratings = []
			for ratingTitle in ratingTitles:
				ratingText = ratingTitle.text
				pos = ratingText.find('(')
				ratingName = ratingText[:pos-1]
				ratingValue = ratingText[pos+1:-1]
				ratings.append({'name': ratingName.title(), 'rating': float(ratingValue)})
			wineInfo['ratings'] = ratings

		wineInfo['name'] = wineInfo['name'].title()
		if 'producer' in wineInfo.keys() and wineInfo['producer'] not in wineInfo['name']:
			wineInfo['name'] = wineInfo['producer'] + ' ' + wineInfo['name']
		if 'vintage' in wineInfo.keys():
			wineInfo['name'] += ' ' + str(wineInfo['vintage'])
		wineInfo['name'] = wineInfo['name'].replace('-', ' ').title()

		print(wineInfo['name'])

		allWines.append(wineInfo)
		# insertWine(wineInfo, conn, cursor)
		

		if wineInfo['name'] == 'Xavier Gerard Cote Rotie La Landonne 2019':
			finish = True

writeToJson(allWines, 'data_bestOfWines2.json')

# conn.close()


Adversity Cabernet Sauvignon Erba Vineyard 2019 Adversity
Albert Bichot Clos De Vougeot Domaine De Clos Frantin 2019 Albert Bichot
Almaviva Puente Alto 2019 Almaviva
Amuse Bouche Leroy Neiman 2019 Amuse Bouche
Amuse Bouche Proprietary Red 2019 Amuse Bouche
Au Sommet Cabernet Sauvignon Atlas Peak 2019 Au Sommet
Beaulieu Vineyard Georges De Latour Private Reserve Cabernet Sauvignon 2019 Beaulieu Vineyard
Beaux Freres Pinot Noir The Belles Soeurs 2019 Beaux Freres
Bibi Graetz Colore 2019 Bibi Graetz
Camarcanda 2019 Camarcanda
Carte Blanche Cabernet Sauvignon Beckstoffer Missouri Hopper Vineyard 2019 Carte Blanche
Carte Blanche The Mark 2019 Carte Blanche
Catena Zapata Adrianna Vineyard Mundus Bacillus Terrae 2019 Catena Zapata
Ceretto Barbaresco Bricco Asili 2019 Ceretto
Cesare Bussolo Barolo Fossati 2019 Cesare Bussolo
Chapoutier Ermitage Greffieux Rouge 2019 Chapoutier
Chapoutier Ermitage Le Meal Rouge 2019 Chapoutier
Chapoutier Le Clos St.Joseph Rouge 2019 Chapoutier
Chateau Bellevue M

NameError: name 'writeToJson' is not defined

In [23]:
conn.close()